In [4]:
# Metrics/Indicators

sharpe_ratio = (dataframe_name.mean()*252) / (dataframe_name.std() * np.sqrt(252))

In [1]:
# Historical Price Vol

# calc daily logarithmic return
dataframe_name['returns'] = (np.log(dataframe_name.close / dataframe_name.close.shift(-1)))

# calc daily std of returns
daily_std = np.std(dataframe_name.returns)

# annualized daily std
std = daily_std * 252 ** 0.5

NameError: name 'np' is not defined

In [3]:
# Implied Vol

In [5]:
# Sortino Ratio